In [1]:
# !rm -rf ln-routing
# !git clone https://github.com/jodobear/ln-routing.git
print('--------------------------------------')
# !rm -rf harbour-space-text-mining-course
# !git clone https://github.com/horoshenkih/harbour-space-text-mining-course.git
import sys
sys.path.append('harbour-space-text-mining-course')

from tmcourse.node2vec import Node2VecGraph

# !python3 -m pip install -U gensim

print('--------------------------------------')
print("Contents of repo: ")
!ls ln-routing/
print('--------------------------------------')
print("Contents of data dir: ")
!ls ln-routing/data
print('--------------------------------------')
print("Contents of res dir: ")
!ls ln-routing/res

--------------------------------------
--------------------------------------
Contents of repo: 
data  ln_graph.ipynb  README.md  res
--------------------------------------
Contents of data dir: 
channels-2020_03_04_02_36_56.csv  nodes-2020_03_04_02_36_56.csv
--------------------------------------
Contents of res dir: 
ln-20-06-04.png  ln-graph.png  ln-routing.jpeg


In [2]:
def hide_code_in_slideshow():   
    from IPython import display
    import binascii
    import os
    uid = binascii.hexlify(os.urandom(8)).decode()    
    html = """<div id="%s"></div>
    <script type="text/javascript">
        $(function(){
            var p = $("#%s");
            if (p.length==0) return;
            while (!p.hasClass("cell")) {
                p=p.parent();
                if (p.prop("tagName") =="body") return;
            }
            var cell = p;
            cell.find(".input").addClass("hide-in-slideshow")
        });
    </script>""" % (uid, uid)
    display.display_html(html, raw=True)

<!--@slideshow slide-->
# Exploring Routing in Lightning Network

Lightning Network(LN) is a second layer payment and authentication protocol on top of the Bitcoin protocol.

![ln-snapshot-20-06-04](https://raw.githubusercontent.com/jodobear/ln-routing/master/res/ln-20-06-04.png)




<!--@slideshow slide-->
## Payment Mechanism

Payments on the LN are routed through channels between participating nodes. Each node can route payments if it has a channel to other node and it isn't necessary to have a direct channel between them if there exists a route between the two nodes.

![ln-routing](https://raw.githubusercontent.com/jodobear/ln-routing/master/res/ln-routing.jpeg)


<!--@slideshow slide-->
## LN Routing Problems

1. Shortest Path Problem — What’s the shortest (cost-wise) path between any two nodes in a graph?

2. Network Flow — Does the directed path have enough capacity to carry the “flow” it receives at each node along the way?

3. Matching Problem — Does a graph contain a pair or more of matching independent edge sets?

4. Critical Path Analysis — In a system of interdependent activities, which is the longest path of a dependent nature?


<!--@slideshow slide-->
# Exploring the Shortest Path Problem

The Network is a directed graph with nodes and channels as edges between nodes.

In this project I am trying to find the shortest path (distance-wise) between a pair of nodes, if it exists.


![ln-graph](https://raw.githubusercontent.com/jodobear/ln-routing/master/res/ln-graph.png)



<!--@slideshow slide-->
## Steps

1. Preprocess the data, fillna, add weight to make it work with Word2Vec.

2. Create a Directed Graph with Networkx.

3. Convert it to a Node2Vec graph and get all the walks in the graph.

4. Train Word2Vec model to build vocabulary(nodes list/words) and get vectors for all the walks(paths/sentences).

5. Compare greedy algorithm implementation vs Networkx shortest path algorithm and analyse the differences.

In [3]:
import sys
sys.path.append('ln-routing')

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx

from pprint import pprint
from tqdm.notebook import tqdm

nodes_df = pd.read_csv('ln-routing/data/nodes-2020_03_04_02_36_56.csv')
channels_df = pd.read_csv('ln-routing/data/channels-2020_03_04_02_36_56.csv')

## Looking At The Data

In [4]:
# Dummy weight is added to make the graph work with Word2Vec
channels_df['weight'] = 1
channels_df.head()

,id,source,target,weight
0,571965x561x1,0288e60c31a434d81e8397231b05229d298bb55885203f...,02d419c9af624d0e7a7c90a60b9ffa35f4934973a9d7d3...,1
1,581757x2079x1,021d7477153209bf1c81060483b4c2519bde173a18cbb3...,03620f31cf2bb95bb70c8c7bcdaa0698cd9bc7f1798729...,1
2,549464x2147x0,03205b045b0126c0d3fb826a38b9befc4babf516895449...,03e50492eab4107a773141bb419e107bda3de3d55652e6...,1
3,575046x2453x1,0230d70b0506aa447a1db407cc04e41448ab92bbd373f5...,0324957a5ab4f045fc02a492eb7a3ff375a4bcec059a6a...,1
4,614549x1353x0,022c10817fd76e01c63198967a77e3a2ac1d051267d809...,0282c03304843150091acd28fbbd2341d73571004ecd48...,1


In [5]:
nodes_df.head()

,id,ip,geo,alias,latency
0,032b31feacb6d645c9a83a015d2b52d549205ef515ddea...,94.130.239.161:9735,"{'latitude': 51.2993, 'longitude': 9.491}",CoinMall.com,0
1,02827a7ba367d10a29f0a178be878f737292889d1926b4...,18.223.138.245:9735,"{'latitude': 39.9653, 'longitude': -83.0235}",SOVEREIGNHOMININ,0
2,02cfe9b37f3fb1012453f6de7e1c2a9aa6e06ff8c91230...,NaN,"{'latitude': 0, 'longitude': 0}",vasek,0
3,025cde2c2b144d49cf960c9292d6b345624753e66f0b3b...,NaN,"{'latitude': 0, 'longitude': 0}",BleeblerBlobbler,0
4,02ca359f3c471db5841d17dc112b10e1a04b8532aab95a...,NaN,"{'latitude': 0, 'longitude': 0}",BER,0


In [6]:
# print(nodes_df.shape, channels_df.shape)
# pprint(nodes_df.isnull().sum())
# print('--------------')
# pprint(channels_df.isnull().sum())
# print('--------------')
# nodes_df = nodes_df.fillna(-1)
# print(nodes_df.isnull().sum())

In [7]:
# This is how you do a multiDi graph but can't work with this and node2vec
# since it stores weight for each edge from a node.
# Hence using directed graph as constructed below.
# G = nx.from_pandas_edgelist(channels_df,
#                             source='source',
#                             target='target',
#                             edge_attr=True,
#                             create_using=nx.MultiDiGraph())
# G.nodes(), G.edges(), G.degree()

## Creating the Graphs

In [8]:
#@slideshow slide
# creating Networkx Graph from df
from collections import Counter

edge2weight = Counter()
for _, r in channels_df.iterrows():
    edge2weight[(r['source'], r['target'])] += 1

edges_for_digraph = [
    (e[0], e[1], {'weight': v})
    for e, v in edge2weight.items()
]

G = nx.DiGraph()
G.add_edges_from(edges_for_digraph)
print("Number of nodes/characters: ", len(G.nodes()))
print("Number of edges/connections: ", len(G.edges()))

Number of nodes/characters:  5026
Number of edges/connections:  28150


In [9]:
# pos = nx.layout.spring_layout(G)
# colors = range(28150)
# options = {
#     "node_color": "#A0CBE2",
#     "edge_color": colors,
#     "width": 4,
#     "edge_cmap": plt.cm.Blues,
#     "with_labels": False,
# }
# nx.draw(G, pos, **options)
# plt.show()

In [10]:
nodes_with_edges = [node for node, degree in dict(G.degree()).items() if degree > 0]

print("Nodes from nodes_df:    ", len(nodes_df['id']))
print("Nodes from G:           ", len(G.nodes()))
print("Nodes with Edges:       ", len(nodes_with_edges))
print("Edges from channels_df: ", len(channels_df))
print("Edges from G:           ", G.number_of_edges())
print("-----")
print("Has Attracting Components:               ", nx.is_attracting_component(G))
print("Strongly Connected:                      ", nx.is_strongly_connected(G))
print("Weakly Connected:                        ", nx.is_weakly_connected(G))
print("Semi Connected:                          ", nx.is_semiconnected(G))
print("Number of Strongly Connected Components: ", nx.number_strongly_connected_components(G))
print("Number of Attracting Components or HUBS: ", nx.number_attracting_components(G))
print("Number of Weakly Connected Components:   ", nx.number_weakly_connected_components(G))

smallest_weakly_cc = min(nx.weakly_connected_components(G), key=len)
print("Smallest Weakly Connected Component:     ", len(smallest_weakly_cc))

Nodes from nodes_df:     4864
Nodes from G:            5026
Nodes with Edges:        5026
Edges from channels_df:  31603
Edges from G:            28150
-----
Has Attracting Components:                False
Strongly Connected:                       False
Weakly Connected:                         False
Semi Connected:                           False
Number of Strongly Connected Components:  5026
Number of Attracting Components or HUBS:  1355
Number of Weakly Connected Components:    8
Smallest Weakly Connected Component:      2


In [11]:
#@slideshow slide 
# creating a node2vec graph and creating walks
node2vec_G = Node2VecGraph(G, p=1, q=1, is_directed=True)
walks = node2vec_G.simulate_walks(num_walks=10, walk_length=10)
print("Total walks:", len(walks))

Total walks: 50260


In [12]:
#@slideshow slide
# creating a vocabulary(nodes) and treating each walk as a sentece with word2vec
from gensim.models import Word2Vec

model = Word2Vec(
    sg=1,  # skip-gram
    hs=0,  # negative sampling
    size=200,
    window=5,
    alpha=0.01,
    negative=20
)
model.build_vocab(walks)
print("Word2Vec vocab: ", len(model.wv.vocab))
model.train(walks, total_examples=len(walks), epochs=10)

Word2Vec vocab:  5026


(1373507, 1869550)

In [13]:
# pprint(model.wv.most_similar("027e62436314f4a18b5cf459d3735977d00830dd3ff24da7f14214824f89fe8d34"))
# set(G.neighbors(list(G)[42]))

## The Algorithms

In [14]:
#@slideshow slide
# implementation of greedy shortest path
def find_greedy_sh_path(G, s, t, model=None):
  """
  G: Networkx graph
  s, t: nodes
  
  return value: list of least # of connected nodes between s and t, inc. s and t (or -1, if no path found)
  """
  def _dist(u, v):
    return model.wv.similarity(u, v)
  
  visited = {s}
  path = []
  current = s
  while current != t:
    # choose the neighbor node to move to
    neighbor_set = (set(G.neighbors(current)) - visited)  # neighbour set
    
    # all neighbors have been visited, the algorithm gets stuck
    if not neighbor_set:
      return -1

    neighbor_set = list(neighbor_set)
    # move to the neighbor closest to the t
    distance_set = [_dist(t, node) for node in neighbor_set]  # distance set
    closest = neighbor_set[np.argmax(distance_set)]
    
    visited.add(current)  # mark left node as visited
    path.append(current)  # update path
    current = closest     # move
  path.append(t)
  return path

In [15]:
#@slideshow slide
# Networkx shortest path
def nx_sh_path(G, s, t, model=None):
  try:
    return nx.shortest_path(G, s, t)
  except nx.NetworkXNoPath:
    return -1

In [16]:
#@slideshow slide
# helper functions
def get_1000_pairs(num_nodes: int) -> list:
  """
  return value: list of 1000 random integer tuples
  """
  import random
  pairs = []
  for _ in range(1000):
    x, y = random.randrange(num_nodes), random.randrange(num_nodes)
    pairs.append((x, y))
  return pairs


def build_paths(fn, G, nodes_list, node_pairs, model=None):
  """
  fn: function to find path
  G: networkx graph
  nodes_list: list of nodes in G
  node_pairs: list of index pairs for nodes_list

  return value: dict of node pair tuple as key and list with path between them as value
  """
  paths = dict()
  for i in range(len(node_pairs)):
    x, y = node_pairs[i][0], node_pairs[i][1]
    s, t = nodes_list[x], nodes_list[y]
    path = fn(G, s, t, model)
    if path != -1:
      paths[(x, y)] = path
  return paths

In [17]:
# functions to investigate paths
def find_common_keys(dictA, dictB):
  """
  dictA, dictB: dictionaries with similar keys

  return value: list of matching node_pair
  """
  matches = []
  for key in dictA.keys():
    if key in dictB.keys():
      matches.append(key)
  return matches


def find_path_diffs(greedy_paths, nx_paths, common_node_pairs):
  """
  greedy_paths, nx_paths: dictionaries of paths
  common_node_pairs: list of tuples of common node pairs in both dicts

  return value: dict with node pair as key and tuple of 2 dicts containing path length from both dicts that differ for the same keys
  """
  paths = dict()
  for i in range(len(common_node_pairs)):
    path_in_greedy = len(greedy_paths[common_node_pairs[i]])
    path_in_nx = len(nx_paths[common_node_pairs[i]])
    if path_in_greedy != path_in_nx:
      paths[common_node_pairs[i]] = ({"greedy_path": path_in_greedy}, {"nx_path": path_in_nx})
  return paths

In [18]:
# more funcitons to investigate paths
def paths_with_neighbors(paths_dict):
  """
  paths_dict: dict with shortest paths

  return value: list of node pairs with path len 2 i.e. containing only s & t nodes
  """
  neighbors = []
  for k, v in paths_dict.items():
    if len(v) == 2:
      neighbors.append(k)
  return neighbors
  print("nx")

def has_shorter_path(paths_dictA, paths_dictB, node_pairs):
  """
  paths_dictA, paths_dictB: dicts with shortest paths
  node_pairs: tuple of pairs of nodes

  return value: Bool, True if paths_dictA has any paths shorter than paths_dictB
  """
  for i in range(len(paths_dictA)):
    if node_pairs[i] in paths_dictA:
      return True if len(paths_dictA[node_pairs[i]]) < len(paths_dictB[node_pairs[i]]) else False

## Analysis of the Paths from Greedy vs Networkx

In [19]:
#@slideshow slide tags=remove_input
hide_code_in_slideshow()
nodes_list = list(G) # 5026
node_pairs = get_1000_pairs(len(nodes_list))
greedy_paths = build_paths(find_greedy_sh_path, G, nodes_list, node_pairs, model=model)
nx_paths = build_paths(nx_sh_path, G, nodes_list, node_pairs, model=model)
common_node_pairs = find_common_keys(greedy_paths, nx_paths)
print("Number of paths found by greedy: ", len(greedy_paths))
print("Number of paths found by networkx: ", len(nx_paths))
print("Number of common node pairs in both: ", len(common_node_pairs))
print("---------------------------------------")
diff_paths = find_path_diffs(greedy_paths, nx_paths, common_node_pairs)
greedy_neighbor_paths = paths_with_neighbors(greedy_paths)
nx_neighbor_paths = paths_with_neighbors(nx_paths)
print("Number of paths with only neighbors:")
print("Greedy Algorithm: ", len(greedy_neighbor_paths))
print("Networkx Algorithm: ", len(nx_neighbor_paths))
print("---------------------------------------")
any_short_path_in_greedy = has_shorter_path(greedy_paths, nx_paths, node_pairs)
print(f"Does greedy algorithm produce a shorter path than Networkx? {any_short_path_in_greedy}")
print("---------------------------------------")
print(f"There are {len(diff_paths)} paths of different lengths, 3 of them are: \n")
for i in list(diff_paths)[:3]:
  print("Node Pairs: ", i, ">>> Greedy Path: ", diff_paths[i][0]['greedy_path'], "| Nx Path: ", diff_paths[i][1]['nx_path'])

Number of paths found by greedy:  29
Number of paths found by networkx:  161
Number of common node pairs in both:  29
---------------------------------------
Number of paths with only neighbors:
Greedy Algorithm:  0
Networkx Algorithm:  0
---------------------------------------
Does greedy algorithm produce a shorter path than Networkx? None
---------------------------------------
There are 8 paths of different lengths, 3 of them are: 

Node Pairs:  (2035, 518) >>> Greedy Path:  6 | Nx Path:  3
Node Pairs:  (154, 736) >>> Greedy Path:  4 | Nx Path:  3
Node Pairs:  (833, 2745) >>> Greedy Path:  6 | Nx Path:  3


In [21]:
pprint(greedy_paths[(154, 736)])
print("---------------------------------------------------------------------")
pprint(nx_paths[(154, 736)])

['02bb24da3d0fb0793f4918c7599f973cc402f0912ec3fb530470f1fc08bdd6ecb5',
 '03271338633d2d37b285dae4df40b413d8c6c791fbee7797bc5dc70812196d7d5c',
 '033d8656219478701227199cbd6f670335c8d408a92ae88b962c49d4dc0e83e025',
 '035abecb83cb67f3f36b2eabb946ffb171a13a82877bd7d3b406588ef89f64e064']
---------------------------------------------------------------------
['02bb24da3d0fb0793f4918c7599f973cc402f0912ec3fb530470f1fc08bdd6ecb5',
 '032679fec1213e5b0a23e066c019d7b991b95c6e4d28806b9ebd1362f9e32775cf',
 '035abecb83cb67f3f36b2eabb946ffb171a13a82877bd7d3b406588ef89f64e064']


## Using the same methodology to analyse GoT Character Graph

In [22]:
#@slideshow slide tags=remove_input
# Training the algorithms on GoT data
hide_code_in_slideshow()
got_df = pd.read_csv("harbour-space-text-mining-course/datasets/got/asoiaf-book5-edges.csv")
got_df.head()

,Source,Target,Type,weight,book
0,Aegon-I-Targaryen,Daenerys-Targaryen,undirected,4,5
1,Aegon-Targaryen-(son-of-Rhaegar),Daenerys-Targaryen,undirected,11,5
2,Aegon-Targaryen-(son-of-Rhaegar),Elia-Martell,undirected,4,5
3,Aegon-Targaryen-(son-of-Rhaegar),Franklyn-Flowers,undirected,3,5
4,Aegon-Targaryen-(son-of-Rhaegar),Haldon,undirected,14,5


In [23]:
#@slideshow slide tags=remove_input
# creating Networkx graph
GG = nx.from_pandas_edgelist(got_df,
                            source='Source',
                            target='Target',
                            edge_attr=True,
                            create_using=nx.Graph())
print("Number of nodes/characters: ", len(GG.nodes()))
print("Number of edges/connections: ", len(GG.edges()))

# creating a node2vec graph and creating walks
node2vec_GG = Node2VecGraph(GG, p=1, q=1, is_directed=False)
gg_walks = node2vec_GG.simulate_walks(num_walks=10, walk_length=10)
print("\nTotal walks:", len(gg_walks))

# creating a vocabulary(nodes) and treating each walk as a sentece with word2vec
gg_model = Word2Vec(
    sg=1,  # skip-gram
    hs=0,  # negative sampling
    size=200,
    window=5,
    alpha=0.01,
    negative=20
)
gg_model.build_vocab(gg_walks)
print("Word2Vec vocab: ", len(gg_model.wv.vocab))
gg_model.train(gg_walks, total_examples=len(gg_walks), epochs=10)

Number of nodes/characters:  317
Number of edges/connections:  760

Total walks: 3170
Word2Vec vocab:  317


(195352, 317000)

## Analysis of GoT graph paths Greedy vs Networkx

In [24]:
#@slideshow slide tags=remove_input
hide_code_in_slideshow()
gg_nodes_list = list(GG)
gg_node_pairs = get_1000_pairs(len(gg_nodes_list))
gg_greedy_paths = build_paths(find_greedy_sh_path, GG, gg_nodes_list, gg_node_pairs, model=gg_model)
gg_nx_paths = build_paths(nx_sh_path, GG, gg_nodes_list, gg_node_pairs, model=gg_model)
gg_common_node_pairs = find_common_keys(gg_greedy_paths, gg_nx_paths)
print("Number of paths found by greedy: ", len(gg_greedy_paths))
print("Number of paths found by networkx: ", len(gg_nx_paths))
print("Number of common node pairs in both: ", len(gg_common_node_pairs))
print("---------------------------------------")
gg_diff_paths = find_path_diffs(gg_greedy_paths, gg_nx_paths, gg_common_node_pairs)
gg_greedy_neighbor_paths = paths_with_neighbors(gg_greedy_paths)
gg_nx_neighbor_paths = paths_with_neighbors(gg_nx_paths)
print("Number of paths with only neighbors:")
print("Greedy Algorithm: ", len(gg_greedy_neighbor_paths))
print("Networkx Algorithm: ", len(gg_nx_neighbor_paths))
print("---------------------------------------")
any_short_path_in_greedy = has_shorter_path(gg_greedy_paths, gg_nx_paths, gg_node_pairs)
print(f"Does greedy algorithm produce a shorter path than Networkx? {any_short_path_in_greedy}")
print("---------------------------------------")
print(f"There are {len(gg_diff_paths)} paths of different lengths, 3 of them are: \n")
for i in list(gg_diff_paths)[:3]:
  print("Node Pairs: ", i, ">>> Greedy Path: ", gg_diff_paths[i][0]['greedy_path'], "| Nx Path: ", gg_diff_paths[i][1]['nx_path'])

Number of paths found by greedy:  756
Number of paths found by networkx:  996
Number of common node pairs in both:  756
---------------------------------------
Number of paths with only neighbors:
Greedy Algorithm:  14
Networkx Algorithm:  14
---------------------------------------
Does greedy algorithm produce a shorter path than Networkx? False
---------------------------------------
There are 277 paths of different lengths, 3 of them are: 

Node Pairs:  (15, 311) >>> Greedy Path:  6 | Nx Path:  5
Node Pairs:  (26, 54) >>> Greedy Path:  5 | Nx Path:  4
Node Pairs:  (44, 258) >>> Greedy Path:  5 | Nx Path:  4


In [25]:
pprint(gg_greedy_paths[(15, 311)])
print("-------------------------")
pprint(gg_nx_paths[(15, 311)])

['Yandry',
 'Jon-Connington',
 'Robert-Baratheon',
 'Eddard-Stark',
 'Theon-Greyjoy',
 'Walder-Frey-(son-of-Jammos)']
-------------------------
['Yandry',
 'Tyrion-Lannister',
 'Stannis-Baratheon',
 'Theon-Greyjoy',
 'Walder-Frey-(son-of-Jammos)']


# Conclusion

- Though I was able to find shortest paths with a greedy algorithm, it is far from optimised. It not only cannot find all the paths, but some of them can be shorter as found by Networkx.

- This was a much simplified problem as there are quite a few considerations in finding an optimal path, some of the variables are Cost, Channel Balances, Hops, Latency.

### Main Challenges

- Understanding how the libraries worked, Networkx & Node2Vec especially since there's virtually no documentation apart from the whitepaper.
- Understanding and visualizing how the data changes when converting the data from one form to the other.

### Learnings

- Even with such cursory analysis I was able to identify loosely connected components in the graph. This is very interesting since, loosely connected components offer a commercial opportunity.

### Next Steps

- Improve Distance measure.
- Perform Clustering, Density analysis and Connectivity Analysis.